## MONGOOSE Prototype 1
### Goal: Implement smart hash-table update scheduler

Maintain two copies of the parameters $W$ (trainable params) and $V$ (lagged copy of V).
Maintain a list $S$ of the nodes $w_i$ in $W$ where $||w_i - v_i|| > \epsilon$ where $\epsilon$ is some error threshold parameter (e.g. $v_i$ are stale). So $S$ tracks the number of node indices that are stale. 

Everytime we query the hash table, we also compute $S$. If $|S| > \rho$ then we initiate an LSH table update of just those stale nodes, and also include nodes that are close to the decision-boundary $\epsilon$ if there are many nodes close to the decision boundary, e.g. $S \leftarrow w_i, if ||w_i - v_i|| > \epsilon / 2 \text{ and } i > \rho$ where $\rho$ is some parameter, e.g. if 5% of the nodes are near the decision boundary then do a full update of those too, because when there are a lot nodes close to the decision-boundary then they are likely to become stale soon anyway.

So every LSH query we compute $S$ and then use that to decide whether to initiate a hashtable update of selected stale nodes. When we update the nodes in $S$, we also update the lagged copy of the parameters $V$ for those nodes so that their synchronized with the main parameters $W$.

Major issue: Training is exploding in memory for unclear reasons. Somehow related to check_stale and updating hash tables. Another issues is that with multithreading it dies; possible due to corrupting the hash tables by simultaneously writing since more updates are happening more frequently

In [ ]:
using Revise
using Flux
using Zygote
using MLDatasets
using LSHFunctions
using DataStructures
using Plots;
using Profile;
using StatProfilerHTML;
using BenchmarkTools
using LinearAlgebra;
using Random;
using Statistics
#using Distributions;
include("./Optim.jl");
using .Optim

In [ ]:
#Parameters
batch_size = 8
hash_len = 5  #k,L= 7,10
n_tables = 10;
bin_size = 30
sample_prop = 0.01 # 5%, proportion of nodes to sample from matrix
update_thresh_eps = 0.05 # minimum difference between lagged vector and trained vector to consider stale, ∈ [0,0.1]
update_thresh_num = 0.1; # percentage of nodes that are stale before triggering update

In [ ]:
function onehotencode(y)
    t = zeros(Float64,10)
    t[y[]+1] = 1.0
    return t
end

In [ ]:
train_x, train_y = MNIST.traindata();
train_x = permutedims(train_x,(3,2,1))
x_train = reshape(train_x,(size(train_x)[1],prod(size(train_x)[2:end])));
y_train = [transpose(onehotencode(y)) for y in train_y]
y_train = vcat(y_train...);
x_train = convert(Array{Float64}, x_train);

In [ ]:
test_x, test_y = MNIST.testdata();
test_x = permutedims(test_x,(3,2,1))
x_test = reshape(test_x,(size(test_x)[1],prod(size(test_x)[2:end])));
y_test = [transpose(onehotencode(y)) for y in test_y]
y_test = vcat(y_test...);
x_test = convert(Array{Float64}, x_test);

In [ ]:
#tid = rand(1:60000)
#println(y_train[tid,:])
#Gray.(reshape(x_train[tid,:],(28,28)))

In [ ]:
l2norm(x) = √(dot(x,x))

In [ ]:
moving_average(vs,n) = [sum(@view vs[i:(i+n-1)])/n for i in 1:(length(vs)-(n-1))]

In [ ]:
function scale(x)
    n = length(x)
    s = size(x)
    r1 = Array{Float64}(undef,n)
    x_ = reshape(x,n)
    mn = minimum(x_)
    r1 = (x_ .- mn)
    mx = maximum(r1)
    r2 = Array{Float64}(undef,n)
    r2 = (r1 ./ mx)
    reshape(r2,s)
end

In [ ]:
[-1,0,5] |> scale

In [ ]:
function standardize(x)
    μ = mean(x)
    σ = std(x)
    r = (x .- μ) ./ σ
end

In [ ]:
function comb(n, len) 
    Iterators.product(fill(BitArray([0,1]), len)...) |> collect |> vec 
end

In [ ]:
all_hash_codes_ = comb(1,hash_len);
all_hash_codes = (all_hash_codes_[:] .|> BitVector);

In [ ]:
mutable struct FixLayer
    theta::Matrix #original
    bias::Matrix #original
    theta_lag::Matrix
    #S::Set{Int64} #stores stale node indices
end

mutable struct LSHLayer
    theta::Matrix #original
    bias::Matrix #original
    theta_lag::Matrix #lagged copy for mongoose
    hash_funs::Vector #Vector{SimHash}
    hash_tables::Vector{Dict{BitVector,CircularBuffer{Integer}}}
    #sampled::Vector
    #S::Set{Int64} #stores stale node indices
end

mutable struct ViewLayer
    thetaV::SubArray
    biasV::SubArray
end

In [ ]:
#The `Layer` type constructor
function LSHLayer(in_dim::Integer,out_dim::Integer,k=6,L=6,bin_size=bin_size)
    div_n = 10
    theta = randn(in_dim,out_dim) / div_n
    bias = randn(1,out_dim) / div_n
    cols = size(theta)[2] #number of columns/nodes
    #hsh = SignALSH(k,maxnorm=5)
    hash_funs = [SimHash(k) for i in 1:L] #LSHFunction(cossim, k)
    hash_tables = Vector{Dict{BitVector,CircularBuffer{Integer}}}()
    for i in 1:L #create L hash tables
        ht_l::Dict{BitVector,CircularBuffer{Integer}} = Dict{BitVector,CircularBuffer{Integer}}((x) => CircularBuffer{Integer}(bin_size) for x in all_hash_codes)
        push!(hash_tables,ht_l)
        for j in 1:cols
            t = theta[:,j]# |> standardize #Flux.normalize(theta[:,j])
            hash_lj = hash_funs[i](t) |> BitVector
            #hash_lj = Tuple(hash_lj)
            push!(hash_tables[i][hash_lj],j)
        end
    end
    return LSHLayer(theta,bias,copy(theta),hash_funs,hash_tables), ViewLayer(view(theta,:,:),view(bias,:,:))
end

function FixLayer(in_dim::Integer,out_dim::Integer)
    div_n = 10
    theta = randn(in_dim,out_dim) / div_n
    bias = randn(1,out_dim) / div_n
    return FixLayer(theta,bias,copy(theta)), ViewLayer(view(theta,:,:),view(bias,:,:))
end

In [ ]:
function sample_nodes(query::Vector, layer::LSHLayer)
    #`query` is the input vector for this layer
    S = Set{Int64}([])
    #num_ht = copy(L)#layers[1].hash_funs |> length
    cols = size(layer.theta)[2]
    maxN::Int64 = (sample_prop * cols) |> round
    x = copy(query)
    while length(S) < maxN
        i = rand(1:n_tables) #get random hash table index
        # compute hash of query using each hashfun
        q_hash = layer.hash_funs[i](x) |> BitVector
        q_hash = rand(0:1,hash_len)
        matches = layer.hash_tables[i][q_hash]
        union!(S,matches)
        if isempty(matches)
            r = rand(1:cols,bin_size)
            union!(S,r)
        end
    end
    out = collect(S)
end

In [ ]:
#=@time begin Threads.@threads for i in 1:100000
    r = rand(1:60000)
    #print(" $r ")
    sample_nodes(x_train[r,:],layers[1])
end
end=#

In [200]:
"""
Input: an LSH layer
Output: vector of node indices

This function will compare the trained parameters and the lagged copy of the parameters to find those nodes that
are stale, i.e. where the lagged copy is significantly different from the main copy.
"""
function check_stale(stale::Set{Int64},layer::LSHLayer)
    
    rows, cols = size(layer.theta)
    notS = setdiff(1:cols,stale)
    maxN::Int64 = (0.1 * rows) |> round
    rid = rand(1:rows,maxN)
    theta = layer.theta[rid,:]
    theta_lag = layer.theta_lag[rid,:]
    td = theta .- theta_lag
    ϵ = update_thresh_eps #0.001 * l2norm(theta) #update_thresh_eps
    S::Set{Int64} = Set{Int64}([])
    S₂::Set{Int64} = Set{Int64}([])
    t = (ϵ * cols) |> round
    close_bound = ( (0.9) * ϵ ) #close to decision boundary
    for i in notS
        a = td[:,i]
        z = l2norm(a)
        if z >= ϵ
            push!(S, i)
        elseif z >= close_bound #if there are a lot of points close to decision boundary
            push!(S₂, i)
        end
    end
    #println(length(S₂))
    if length(S₂) > (1.5 * t)
        S = (S ∪ S₂)
    end
    R = copy(stale ∪ S)
end

check_stale

In [158]:
#s_ = Set{Int64}([])
#S2 = check_stale(s_,layers[1])

In [ ]:
function get_view(a::Matrix,r,c)
    #r,c are arrays, if empty [] then return all elements
    d1,d2 = size(a);
    return view(a, isempty(r) ? (:) : r, isempty(c) ? (:) : c)
end

In [ ]:
# Define the function that runs an LSH layer
function (m::LSHLayer)(X::Matrix,rows::Vector,V::ViewLayer)
    Zygote.ignore() do
        cols = sample_nodes(vec(X),m) #rand(1:size(m.theta)[2],90)
        cols = sort(cols);
        V.thetaV = get_view(m.theta,rows,cols)
        V.biasV = get_view(m.bias,[],cols)
    end
    y = X * V.thetaV .+ V.biasV
end
#no activation function applied yet

function (m::FixLayer)(X::Matrix,rows::Vector,V::ViewLayer)
    Zygote.ignore() do
        V.thetaV = get_view(m.theta,rows,[])
        V.biasV = get_view(m.bias,[],[])
    end
    y = X * V.thetaV .+ V.biasV
end

# <span style="color:red">Initialize Layers</span>

In [201]:
dim1,dim2,dim3 = 784, 1000, 10
layer1, layer1view = LSHLayer(dim1,dim2, hash_len, n_tables) #k,L
layer2, layer2view = FixLayer(dim2,dim3)
layers = [layer1, layer2];
layerviews = [layer1view, layer2view];

In [ ]:
function get_view_indices(layerview)
   return layerview.thetaV.indices[2] 
end

In [ ]:
#get_view_indices(layerviews[1])

In [164]:
function model(X::Matrix,layers::Vector,layerviews::Vector)
    #layer 1
    X = Flux.normalise(X;dims=ndims(X), ϵ=1e-5) #
    A1 = (layers[1](X,Vector{Integer}[],layerviews[1]))
    A1 = NNlib.relu.(A1)
    #layer 2
    A2 = Flux.normalise(A1;dims=ndims(A1), ϵ=1e-5)
    A2 = (layers[2](A2,get_view_indices(layerviews[1]),layerviews[2]))#layers[1].sampled
    A2 = NNlib.softmax(A2,dims=2)
end

model (generic function with 1 method)

In [ ]:
lossfn(ŷ::Vector,y::Vector) = -1.0 * LinearAlgebra.dot(log.(ŷ),y)

function lossfn2(x::Matrix,y::Vector,layers::Vector,layerviews::Vector)
    ŷ = vec(model(x, layers, layerviews));
    l = -1.0 * LinearAlgebra.dot(log.(ŷ),y)
end

In [ ]:
function update_htables(S::Set{Int64},ms::Vector)
    u = 0
    for layer in ms
        num_ht = n_tables #length(layer.hash_funs)
        cols = size(layer.theta)[2]
        ns = length(S)
        t = (update_thresh_num * cols) |> round
        if ns >= t
            u += 1
            while !isempty(S)
                j = pop!(S)
                for i in 1:num_ht #iterate tables
                    z = layer.theta[:,j]# |> standardize
                    hash_lj = layer.hash_funs[i](z) |> BitVector
                    #hash_lj = Tuple(hash_lj)
                    push!(layer.hash_tables[i][hash_lj],j)
                    layer.theta_lag[:,j] = copy(layer.theta[:,j])
                end
            end
        end
    end
    
    return S,u
end

In [ ]:
lossfn(ŷ::Vector,y::Vector) = -1.0 * LinearAlgebra.dot(log.(ŷ),y)

function lossfn2(x::Matrix,y::Vector,layers::Vector,layerviews::Vector)
    ŷ = vec(model(x, layers, layerviews));
    l = -1.0 * LinearAlgebra.dot(log.(ŷ),y)
end

In [ ]:
#S = [1,50,90,112,145,240,300,301,500,505,506,511];
#g = Zygote.gradient(w -> lossfn(vec(model(randn(1,784),w,S)),[1.0,0,0,0,0,0,0,0,0,0]),layers)
#g = Zygote.gradient((ypred,ytrue) -> lossfn(ypred,ytrue),(A2,batch_y))
#println(g);

In [ ]:
function get_params_view(layerviews::Vector)
    #idx: [rows1,cols2,rows2,cols2]
    p = []
    #i = 0
    for m in layerviews
        #theta = get_view(m.theta,m.rows,m.cols)
        #bias = get_view(m.bias,[],m.cols)
        append!(p,[m.thetaV,m.biasV])
        #i+=1
    end
    return p
end

In [ ]:
function convert_grads(gs)
    g = []
    for l in gs[1]
        append!(g,[l[][:thetaV],l[][:biasV]])
    end
    g
end

In [ ]:
model(randn(1,784), layers, layerviews)
#w/ scale 26.083 μs (94 allocations: 21.12 KiB)

In [ ]:
#lossfn2(randn(1,784),y_train[1,:],layers,layerviews)

In [ ]:
function gen_layerviews(layers)
    V = []
    for l in layers
        push!(V, ViewLayer(view(l.theta,:,:),view(l.bias,:,:)))
    end
    V
end

In [ ]:
function get_batch(x_train,y_train,batch_size=1)
    rid = rand(1:60000,batch_size)
    x = float(copy(x_train[rid,:]))
    x = reshape(x,(1,784))
    yt = copy(y_train[rid,:])
    yt = reshape(yt,(10))
    return x,yt
end

In [ ]:
#xt,yt = get_batch(x_train,y_train)

In [193]:
function train(x_train,y_train,layers,epochs=200000,lr=0.5)
    opt = SGDM(lr, 0.9, [])#ADAM(lr)
    #ps = get_params_view(layerviews);
    #opt = Optim.ADAM(ps, lr)
    lossarr::Vector{Float64} = []
    updates = 0
    #lk = ReentrantLock()
    S::Set{Int64} = Set([])
    Threads.@threads for i in 1:epochs #Threads.@threads 
        xt,yt = get_batch(x_train,y_train)
        layerviews = gen_layerviews(layers)
        l, gs = withgradient((mv) -> lossfn2(xt,yt,layers,mv), layerviews)
        gs = convert_grads(gs)
        ps = get_params_view(layerviews);
        step!(opt,ps,gs)
        S = check_stale(copy(S),layers[1])
        #print(" $(length(S)) ")
        #(i % 50 == 0) ? print(" $(length(S)) ") : false
        S, u = update_htables(S, layers[1:1])
        updates += u
        #=
        lock(lk) do
            #do something
            push!(lossarr,l)
        end
        =#
    end
    println("Num hash table updates: $updates")
    return lossarr
end

train (generic function with 3 methods)

In [ ]:
#=@profilehtml for i = 1:100
    l, gs = withgradient((mv) -> lossfn2(randn(1,784),randn(10),layers,mv), layerviews)
end=#

In [202]:
@time train(x_train,y_train,layers,25000,0.1)

Num hash table updates: 1017
 16.615596 seconds (132.81 M allocations: 93.974 GiB, 24.71% gc time, 0.72% compilation time)


Float64[]

In [ ]:
#=@time begin
    lossarr = train(x_train,y_train,layers,25000);
end=#
# ~141 seconds for 250k iter with 2-layers and dim2 = 1000 
# 72 seconds for 25k iter
# 32.9 seconds 25k no updates
# 56 seconds 25k w/ updates 63 ht updates

In [ ]:
#plot(moving_average(lossarr,5000)) #y top is > 3, x right is 2 x 10^5
#plot(lossarr[1:2250])

In [ ]:
function test_acc(xs,ys)
    ncorr = 0
    ntot = size(ys)[1]
    #lk = ReentrantLock()
    for i in 1:ntot
        #rid = rand(1:size(xs)[1])
        x = float(xs[i,:])
        x = reshape(x,(1,784))
        yt = ys[i,:]
        ŷ = vec(model(x, layers, layerviews));
        if argmax(ŷ) == argmax(yt)
            ncorr += 1
        end
    end
    println(100 * (ncorr/ntot))
end

In [203]:
test_acc(x_test,y_test)

68.44


# Archive

### Baseline

In [ ]:
#base_model = Chain(Dense(28^2, dim2, relu), Flux.normalise, Dense(dim2, 10), softmax)

In [ ]:
#base_model(x_train[1,:])

In [ ]:
#collapse
#=
function base_train(epochs=100)
    baseopt = Flux.ADAM(0.0001); 
    for i in 1:epochs
        rid = rand(1:60000)
        x = float(x_train[rid,:])
        #x = reshape(x,(1,784))
        yt = y_train[rid,:]
        ps = Flux.params(base_model)
        gs = gradient(ps) do
            ŷ = base_model(x)
            lossfn(ŷ,yt)
        end
        Flux.update!(baseopt,ps,gs)
    end
end
=#

In [ ]:
#=@time begin
    base_train(25000)
end=#
#25k iter: 111 seconds to 95% acc, max acc is 97.5 (if train to convergence)

In [ ]:
#=function base_test_acc(xs,ys)
    ncorr = 0
    ntot = size(ys)[1]
    for i in 1:ntot
        #rid = rand(1:size(xs)[1])
        x = float(xs[i,:])
        #x = reshape(x,(1,784))
        yt = ys[i,:]
        ŷ = vec(base_model(x));
        if argmax(ŷ) == argmax(yt)
            ncorr += 1
        end
    end
    println(100 * (ncorr/ntot))
end=#

In [ ]:
#base_test_acc(x_test,y_test)